<a href="https://colab.research.google.com/github/nhabt-uit/CS2225.CH1801.N014/blob/NhaBT/CNN_Classification_of_Traffic_Signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pickle
import random

with open("/gdrive/My Drive/Colab Notebooks/train.p", mode='rb') as training_data:
  train = pickle.load(training_data)
with open("/gdrive/My Drive/Colab Notebooks/valid.p", mode='rb') as validation_data:
  valid = pickle.load(validation_data)
with open("/gdrive/My Drive/Colab Notebooks/test.p", mode='rb') as testing_data:
  test = pickle.load(testing_data)

X_train, y_train = train['features'], train['labels']
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

width = 7
length = 7

fig, axes = plt.subplots(length, width, figsize = (10,10))
axes = axes.ravel()
ltrain = len(X_train)

for i in np.arange(0, width * length):
  index = np.random.randint(0, ltrain)

  axes[i].imshow(X_train[index])
  axes[i].set_title(y_train[index], fontsize = 1)
  axes[i].axis('off')
plt.subplots_adjust(hspace=0.3)

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

X_train_gray = np.sum(X_train/3, axis=3, keepdims=True)
X_test_gray = np.sum(X_train/3, axis=3, keepdims=True)
X_validation_gray = np.sum(X_train/3, axis=3, keepdims=True)

X_train_gray_norm = (X_train_gray - 128)/128
X_test_gray_norm = (X_train_gray - 128)/128
X_validation_gray_norm = (X_train_gray - 128)/128

#X_train_gray_norm

i = random.randint(1, len(X_train_gray))
plt.imshow(X_train_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_train[i])
plt.figure()
plt.imshow(X_train_gray_norm[i].squeeze(), cmap='gray')

**Building a CNN Model**

In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from subprocess import check_output
from sklearn.model_selection import train_test_split

image_shape = X_train_gray[i].shape
image_shape

cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(32, 3, 3, input_shape = image_shape, activation='relu'))
cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
cnn_model.add(tf.keras.layers.Dropout(0.25))
cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(tf.keras.layers.Dense(128, activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.5))
cnn_model.add(tf.keras.layers.Dense(43, activation='sigmoid'))

cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False),metrics =['accuracy'])

history=cnn_model.fit(X_train_gray_norm,
                      y_train,
                      batch_size=32,
                      epochs=60,
                      verbose=1,
                      validation_data=(X_validation_gray_norm, y_validation))

score = cnn_model.evaluate(X_test_gray_norm, y_test,verbose=0)
print('Test Accuracy : {:.4f}'.format(score[1]))

Epoch 1/60
1086/1088 [============================>.] - ETA: 0s - loss: 2.1671 - accuracy: 0.3900

ValueError: ignored